# Notebook 02: Generation Capacity EDA

In this one we'll load in the cleaned data and just play around with it to see what we can learn

In [1]:
import pandas as pd
from src.data.load_data import GenerationCapacity
from pandas_profiling import ProfileReport
import altair as alt
import altair_data_server

alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [2]:
df = GenerationCapacity().load_cleaned()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25493 entries, 0 to 25799
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   technology             25493 non-null  category
 1   source                 25493 non-null  category
 2   source_type            25493 non-null  category
 3   year                   25493 non-null  int64   
 4   country                25493 non-null  category
 5   capacity               25493 non-null  float64 
 6   energy_source_level_0  25493 non-null  int64   
 7   energy_source_level_1  25493 non-null  int64   
 8   energy_source_level_2  25493 non-null  int64   
 9   energy_source_level_3  25493 non-null  int64   
 10  technology_level       25493 non-null  int64   
dtypes: category(4), float64(1), int64(6)
memory usage: 1.7 MB
